In [ ]:
%pip install langgraph

In [12]:
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
import time
from langgraph.checkpoint.memory import MemorySaver

class ChatState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

def chatbot_responder(state: ChatState):
    print("--- Executing Node: chatbot_responder ---")
    time.sleep(1)
    last_message = state['messages'][-1].content
    if "hello" in last_message.lower():
        response = AIMessage(content="Hello! How can I help you today?")
    elif "langgraph" in last_message.lower():
        response = AIMessage(content="LangGraph is a library for building stateful, multi-actor applications with LLMs.")
    else:
        response = AIMessage(content="I'm not sure how to respond to that. Please ask about 'hello' or 'LangGraph'.")
    return {"messages": [response]}

workflow = StateGraph(ChatState)
workflow.add_node("chatbot", chatbot_responder)
workflow.set_entry_point("chatbot")
workflow.set_finish_point("chatbot")

# Add a checkpointer for state persistence
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("="*50)
print("GRAPH API - STREAMING DEMO")
print("="*50)

config = {"configurable": {"thread_id": "user-thread-1"}}
initial_input = {"messages": [HumanMessage(content="Hello there!")]}

print(f"\n---> Invoking with initial input: {initial_input['messages'][0].content}\n")

for event in app.stream(initial_input, config=config, stream_mode="values"):
    print(f"Event Received: {event}")
    print("-" * 20)
    time.sleep(1)

print("\n---> Checking current state of the graph thread:\n")
current_state = app.get_state(config)
print(f"Current State: {current_state}")

print("\n---> Invoking with a second input to the same thread:\n")
second_input = {"messages": [HumanMessage(content="Tell me about LangGraph.")]}

for event in app.stream(second_input, config=config, stream_mode="values"):
    print(f"Event Received: {event}")
    print("-" * 20)
    time.sleep(1)

print("\n---> Checking final state of the graph thread:\n")
final_state = app.get_state(config)
print(f"Final State: {final_state}")
print("="*50)

GRAPH API - STREAMING DEMO

---> Invoking with initial input: Hello there!

Event Received: {'messages': [HumanMessage(content='Hello there!', additional_kwargs={}, response_metadata={}, id='de9aebb5-16a6-46b3-b7be-cc4e616df4ab')]}
--------------------
--- Executing Node: chatbot_responder ---
Event Received: {'messages': [HumanMessage(content='Hello there!', additional_kwargs={}, response_metadata={}, id='de9aebb5-16a6-46b3-b7be-cc4e616df4ab'), AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}, id='5f76467e-d490-4fe7-a468-c58dd03ed210')]}
--------------------

---> Checking current state of the graph thread:

Current State: StateSnapshot(values={'messages': [HumanMessage(content='Hello there!', additional_kwargs={}, response_metadata={}, id='de9aebb5-16a6-46b3-b7be-cc4e616df4ab'), AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}, id='5f76467e-d490-4fe7-a468-c58dd03ed210')]}, next=(), config=